<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Generators/Upload_Processor_Hema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re
client = OpenAI(api_key="")

In [ ]:
fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-109276.csv")
fda_df = fda_df.sort_values(by=["Active Ingredient UNII(s)","SPL Effective Date (Month/Day/Year)"],ascending=[True,False])
import pandas as pd
import re
all_fda_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/fdalabel-query-378064.csv")
all_fda_df["SPL Effective Date (Month/Day/Year)"] = pd.to_datetime(all_fda_df["SPL Effective Date (Month/Day/Year)"])
all_fda_df["lower"] = all_fda_df["Generic/Proper Name(s)"].str.lower()
all_fda_df = all_fda_df.sort_values(by=["lower","SPL Effective Date (Month/Day/Year)"],ascending=[True,False])
all_fda_df["Active Ingredient(s)"] = all_fda_df["Active Ingredient(s)"].astype(str).dropna()
all_fda_df["Route(s) of Administration"] = all_fda_df["Route(s) of Administration"].astype(str).dropna()
all_fda_df["Active Ingredient UNII(s)"] = all_fda_df["Active Ingredient UNII(s)"].astype(str).dropna()
all_fda_df = all_fda_df[~all_fda_df['Active Ingredient(s)'].str.contains('and|;', regex=True, flags=re.IGNORECASE)]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("TOPICAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("IRRIGATIONAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("INTRADERMAL")]
all_fda_df = all_fda_df[~all_fda_df['Route(s) of Administration'].str.contains("INHALATION")]
all_fda_df = all_fda_df[~all_fda_df['Active Ingredient UNII(s)'].str.contains(";")]
all_fda_df = all_fda_df[all_fda_df["Marketing Category"].isin(["NDA","ANDA","BLA"])]
all_fda_df = all_fda_df.groupby(["lower","Marketing Category"]).first().reset_index()
all_fda_df = all_fda_df.sort_values(["lower","Marketing Category"],ascending=[True,False])
our_df = all_fda_df.groupby("lower").first().reset_index()

In [ ]:
fda_df["Direct FDALabel Link"] = fda_df["FDALabel Link"]
fda_df["Direct Set ID"] = fda_df["SET ID"]
fda_df["lower"] = fda_df["Generic/Proper Name(s)"].str.lower()
fda_df = fda_df[["lower","Direct FDALabel Link","Direct Set ID"]]
our_df = our_df.merge(fda_df,on="lower",how="left").groupby("lower").first().reset_index()
our_df.loc[our_df["Direct FDALabel Link"].isna(),"Direct FDALabel Link"] = our_df.loc[our_df["Direct FDALabel Link"].isna(),"FDALabel Link"]
our_df.loc[our_df["Direct Set ID"].isna(),"Direct Set ID"] = our_df.loc[our_df["Direct Set ID"].isna(),"SET ID"]

In [ ]:
our_df.to_csv("/content/drive/MyDrive/ClinicalTrials/spl_matches_check.csv")

In [ ]:
df_to_use = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/dailymed_full_text_final.csv")

In [ ]:
#USE ME

drugs_list = []
drugs_merged_list = []
drugs_initial_responses = []
drugs_no_less_most_responses = []
drugs_yes_no_responses = []
answers_list = []
initial_prompts = []
no_less_most_prompts = []
yes_no_prompts = []
moiety_list = []
fda_label_list = []
set_list = []

for index, row in tqdm(df_to_use.iterrows(), total=df_to_use.shape[0]):
    #url=url.replace("https://fdalabel.fda.gov:8443","https://nctr-crs.fda.gov")
    #if "nctr-crs.fda.gov" in url:
    #print(plain_text)
    #elif "accessdata" in url:
    #    r = requests.get(url)
    #    f = io.BytesIO(r.content)

    #    reader = PyPDF2.PdfReader(f)
    #    plain_text = "\n".join([reader.pages[i].extract_text().strip() for i in range(0,len(reader.pages))])
    #    plain_text=plain_text.strip()



    initial_prompt = """Provide a summary of all the parts of the drug label that discuss hematological toxicity risks and hematological reactions for this drug.
    In your summary of each sentence, clearly state whether the drug itself was associated with or caused the hematological risk."""


    no_less_most_prompt = """Given the above information about a drug, answer 'was this drug associated with No Hematological Toxicity, Less Hematological Toxicity, or Most Hematological Toxicity?'
        Now, answer with just one word: No, Less or Most"""
    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Hematological Toxicity?'
        Now, answer with just one word: Yes or No"""
    initial_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Read through the following drug label"},
        {"role": "user", "content":row["full_text"]},
        {"role": "user", "content":initial_prompt}
        ],
        stream=False, temperature=0,max_tokens=2000
        )
    no_less_most_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":no_less_most_prompt}
        ],
        stream=False, temperature=0

        )
    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    print(no_less_most_response.choices[0].message.content.strip()," ",yes_no_response.choices[0].message.content.strip())
    drugs_list.append(row["generic_name"])
    moiety_list.append(row["moiety"])
    drugs_initial_responses.append(initial_response.choices[0].message.content.strip())
    drugs_no_less_most_responses.append(no_less_most_response.choices[0].message.content.strip())
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    initial_prompts.append(initial_prompt)
    no_less_most_prompts.append(no_less_most_prompt)
    yes_no_prompts.append(yes_no_prompt)
    fda_label_list.append(url)
    set_list.append(row["set_id"])

    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"urls":fda_label_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_Hema_direct_4o_hematoxicity_temp_"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"urls":fda_label_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_Hema_direct_4o_hematoxicity_final.csv")

In [ ]:
print(final_df["less_determination"].value_counts())

In [ ]:
print(final_df["less_determination"].value_counts())

In [ ]:
zaniar_medications = [
    "FLOXURIDINE",
    "OXAPROZIN",
    "HUMAN SECRETIN",
    "LEFLUNOMIDE",
    "RIFAMPIN",
    "INDIGOTINDISULFONATE SODIUM",
    "LAMOTRIGINE CHEWABLE DISPERSIBLE",
    "IVOSIDENIB",
    "NITISINONE",
    "DEFERASIROX",
    "FERRIC PYROPHOSPHATE CITRATE",
    "SOLRIAMFETOL",
    "TECHNETIUM TC-99M SESTAMIBI",
    "DALFAMPRIDINE",
    "TIMOLOL MALEATE OPHTHALMIC GEL FORMING SOLUTION 0.25%; TIMOLOL MALEATE OPHTHALMIC GEL FORMING SOLUTION 0.5%",
    "MIRTAZAPINE",
    "CIMETIDINE HYDROCHLORIDE ORAL SOLUTION",
    "EPINEPRINE",
    "BREXPIPRAZOLE",
    "SONIDEGIB",
    "PRUSSIAN BLUE INSOLUBLE CAPSULES",
    "DEMECLOCYCLINE HYDROCHLORIDE",
    "DOXYCYCLINE HYCLATE",
    "INEBILIZUMAB",
    "METYROSINE",
    "TOLTERODINE TARTARATE; TOLTERODINE TARTRATE",
    "INSULIN LISPRO-AABC",
    "RALOXIFENE",
    "ACYCLOVIR",
    "LEVETIRACETAM IN SODIUM CHLORIDE",
    "MITOTANE",
    "MORPHINE SULFATE",
    "GALSULFASE",
    "DOLUTEGRAVIR SODIUM",
    "GLUTAMINE",
    "MYCOPHENOLATE MOFETIL",
    "DESIPRAMINE HYDROCHLORIDE",
    "AFATINIB",
    "HEPARIN SODIUM IN SODIUM CHLORIDE",
    "TOLTERODINE",
    "METHAZOLAMIDE",
    "TECLISTAMAB",
    "RALTEGRAVIR",
    "DIMENHYDRINATE",
    "ADUCANUMAB",
    "POTASSIUM CITRATE",
    "GLIPIZIDE ER",
    "PITAVASTATIN",
    "EPIRUBICIN HYDROCHLORIDE",
    "PIROXICAM"
]


In [ ]:
import pandas as pd
final_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/uniTox Ratings/Sunday_Hema_direct_4o_hematoxicity_final.csv")

In [ ]:
zaniar_df = final_df[final_df["generic_name"].isin(zaniar_medications)]

In [ ]:
zaniar_df = zaniar_df[["generic_name","reasoning","less_determination","binary_determination"]].reset_index().drop("index",axis=1)


In [ ]:
zaniar_df.to_csv("/content/drive/MyDrive/ClinicalTrials/zaniar_df.csv")

In [ ]:
angela_medications = [
    "CORTICORELIN OVINE TRIFLUTATE",
    "FLUDEOXYGLUCOSE F-18",
    "PRALSETINIB",
    "BUFFERED PENICILLIN G POTASSIUM",
    "GUSELKUMAB",
    "SUMATRIPTAN SUCCINATE U-D",
    "IMATINIB",
    "IODINATED I-125 ALBUMIN",
    "FLAVOXATE HYDROCHLORIDE",
    "TETRACYCLINE HYDROCHLORIDE",
    "VORTIOXETINE",
    "CHLORHEXIDINE GLUCONATE, 0.12% ORAL RINSE",
    "TECHNETIUM TC 99M MERTIATIDE",
    "AMOXICILLIN TABLET, FILM COATED",
    "OXYMORPHONE HYDROCHLORIDE",
    "GLOFITAMAB",
    "BRINCIDOFOVIR",
    "DIMERCAPROL",
    "ESKETAMINE HYDROCHLORIDE",
    "DEFIBROTIDE SODIUM",
    "LIFITEGRAST",
    "RUXOLITINIB",
    "COAGULATION FACTOR VIIA RECOMBINANT HUMAN",
    "ABACAVIR",
    "0.9% SODIUM CHLORIDE",
    "PERPHENAZINE",
    "MEROPENEM",
    "DESLORATADINE",
    "ROSUVASTATIN CALCIUM; ROSUVASTATIN CALCIUN",
    "BUPIVACAINE HYDROCHLORIDE IN DEXTROSE",
    "CRIZOTINIB",
    "TAFENOQUINE SUCCINATE",
    "SELUMETINIB",
    "LOFEXIDINE HYDROCHLORIDE",
    "DACOMITINIB",
    "TISOTUMAB VEDOTIN",
    "NIMODIPINE",
    "PLERIXAFOR",
    "PARATHYROID HORMONE",
    "CHLORZOXAZONE",
    "CLOMIPRAMINE HYDROCHLORIDE",
    "FLUPHENAZINE DECANOATE",
    "RESLIZUMAB",
    "BUROSUMAB",
    "0.25% BUPIVACAINE HCL; BUPIVACAINE HYDROCHLORIDE",
    "AXITINIB",
    "CIPROFLOXACIN",
    "NIVOLUMAB",
    "ISOSORBIDE MONONITRATE",
    "DILTIAZEM HCI"
]


In [ ]:
import pandas as pd
final_df = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/uniTox Ratings/Sunday_Hema_direct_4o_hematoxicity_final.csv")

In [ ]:
angela_df = final_df[final_df["generic_name"].isin(angela_medications)]

In [ ]:
angela_df = angela_df[["generic_name","reasoning","less_determination","binary_determination"]].reset_index().drop("index",axis=1)


In [ ]:
angela_df.to_csv("/content/drive/MyDrive/ClinicalTrials/angela_hema_df.csv")

In [ ]:
angela_df